# Produce random files in two steps

Thos notebook shows how the random sampling process can be interrupted after the generation of the perturbation coefficients and then resumed in a later stage.

This is done by storing the perturbation coefficients into excel files.
The writing process and the creation of the excel files is done automatically by `sandy`.
Still we can completely stop the pipeline using the command line argument `--only_perturbations` (not active by default).

The advantages of doing so are multiple:
- we can generate an extremely large number of perturbation coefficients (thousands) in a relatively short time and without the need for parallelization;
- since the random files are not created, not only we save computer time, but also disk space (the excel files are very small);
- the perturbation coefficients can be used to ensure reproducibility in the random files;
- also, the perturbation coefficients do not need to be used in their entirety, but can be split into blocks that are more manageable in terms of computer resources - e.g., random files can be created sequentially in chunks of 100;
- having available perturbation coefficients opens up possibilities in terms of sensitivity analysis.

In [ ]:
import sandy

This examples uses the JEFF-3.3 Pu-241 data that contain covariance matrices for both cross sections and fission neutron multiplicities.  

In [ ]:
file = "942410.jeff33"
sandy.get_endf6_file("jeff_33", "xs", 942410).to_file(file)

This is the standard pipeline to produce random files (in this case 2).

In [ ]:
cl = "942410.jeff33 --samples 2 --seed33 1 --seed31 1 --mt33 2"
sandy.sampling.run(cl.split())

Now, let's interrupt the process after that the perturbations are created (reproducible with fixed seed).
This is done by adding the keyword `--only_perturbations`.
The pipeline stops after writing the perturbation coefficients to excel file.

In [ ]:
cl = "942410.jeff33 --samples 2 --seed33 1 --seed31 1 --mt33 2 --only_perturbations"
smps = sandy.sampling.run(cl.split())

Using the perturbation coefficients from the excel files we can generate the same random files of the standard pipeline.
This is done with keyword `--from_perturbations ./ 1 1"`, where:
- `./` is the directory where the perturbation files are lcoated;
- `1 1` are the limits of the sample range to consider, e.g., in this case take only the sample number 1 (this number is reported in the random file name). `0 1` would have taken all sets of perturbation coefficients created in this sequence (in this case 2). If we had 100 sets of perturbation coefficients we could get them all with `0 99`, only the first batch of 50 with `0 49`, or the second batch of 50 with `50 99`.

In [ ]:
cl = "942410.jeff33 --from_perturbations ./ 1 1"
sandy.sampling.run(cl.split())